In [1]:
import glob

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
#import cartopy.crs as ccrs*
import numpy as np
#from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import seaborn as sns
import xarray as xr

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader
#from utils.saverRestorer import SaverRestorer. Hvordan fikser jeg dette Severin

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
# Hyperparameters useful stuff
num_epochs = 5
batch_size = 100
learning_rate = 0.001

DATA_PATH = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/'
MODEL_STORE_PATH = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/storedModel/'

In [4]:
np.logspace(-8, 1, num = 9, endpoint = True )

array([1.00000000e-08, 1.33352143e-07, 1.77827941e-06, 2.37137371e-05,
       3.16227766e-04, 4.21696503e-03, 5.62341325e-02, 7.49894209e-01,
       1.00000000e+01])

In [5]:
def create_ML_repo(start = "2012-01-01", stop = "2013-01-01", season = "SON", 
                   era_path = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/'):
    """ Want all year put season to. """
    q   = xr.open_dataset(glob.glob(era_path + "*_q_*" + season +".nc")[0]).q.values
    r   = xr.open_dataset(glob.glob(era_path + "*_r_*"+ season +".nc")[0]).r.values
    tcc = xr.open_dataset(glob.glob(era_path + "*tcc*"+ season +".nc")[0]).tcc.values
    sp  = xr.open_dataset(glob.glob(era_path + "*sp*"+ season +".nc")[0]).sp.values
    t2m = xr.open_dataset(glob.glob(era_path + "*t2m*"+ season +".nc")[0]).t2m.values
    assert np.shape(q) == np.shape(r) == np.shape(tcc) == np.shape(sp) == np.shape(t2m)
    
    nbr_times, nbr_lats, nbr_lon = np.shape(q)
    train = []
    true  = tcc # is wrong now
    a, b, c = np.shape(true)
    true = true.reshape(a, 1, b, c)
    
    for i in range(nbr_times):
        one_timestep = np.array([q[i], r[i], sp[i], t2m[i] ])
        train.append(one_timestep)

    return np.array(train), true

In [6]:
def train_test_split(data, test_split = 0.2):
    assert test_split < 1, 'test split is given as a decimal number, choose a number between 0, 1'
    indx = int(len(data)*(1-test_split))
    # returns train test
    return data[:indx], data[indx:]

In [29]:
#train, test = train_test_split(np.concatenate((train, test), axis = 1))

In [ ]:
keras.layers.ConvLSTM2DCell(filters, kernel_size, strides=(1, 1), padding='valid', 
                            data_format=None, dilation_rate=(1, 1), activation='tanh', 
                            recurrent_activation='hard_sigmoid', use_bias=True, 
                            kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
                            bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, 
                            recurrent_regularizer=None, bias_regularizer=None, kernel_constraint=None, 
                            recurrent_constraint=None, 
                            bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)

In [8]:
#train = train.reshape(546, 4, 4, 35, 60)

In [9]:
#train

In [10]:
#next(iter(train_loader)).shape

In [11]:
#imgs, steering_angle = next()

In [17]:
loss_fn = torch.nn.MSELoss()
convlstm = ConvLSTM(input_channels = 4, hidden_channels = [256, 128, 64, 1], kernel_size = 5).cpu()
optimizer = optim.RMSprop(convlstm.parameters(), lr=0.001, momentum=0.9)
train, test = create_ML_repo()
train_loader = DataLoader(dataset=np.concatenate((train, test), axis = 1), batch_size=32)

In [18]:
convlstm.apply(init_weights)

ConvLSTM(
  (cell0): ConvLSTMCell(
    (Wxi): Conv2d(4, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (Whi): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
    (Wxf): Conv2d(4, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (Whf): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
    (Wxc): Conv2d(4, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (Whc): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
    (Wxo): Conv2d(4, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (Who): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
  )
  (cell1): ConvLSTMCell(
    (Wxi): Conv2d(256, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (Whi): Conv2d(128, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
    (Wxf): Conv2d(256, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (Whf): Conv

In [ ]:
score = []
nbr_epochs = 10

for epoch in range(nbr_epochs):  
    # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(iter(train_loader)):
        # get the inputs; data is a list of [inputs, labels]
        
        input = data[:, :-1, :, :]
        target = data[:, -1, :, :]
        a,b,c = target.shape
        target = target.reshape(a, 1, b, c)
        
        inputs = torch.tensor(input)
        clouds = torch.tensor(target)
        
        # Shape of input [samples (nbr days), nr hours, lat, lon, metvars]
        input_  = Variable(inputs, requires_grad=True).cpu()
        target_ = Variable(clouds, requires_grad=True).double().cpu()
        
        #output = output[0][0].double()
        #res = torch.autograd.gradcheck(loss_fn, (output, target_), eps=1e-6, raise_exception=True)
        #print(res)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs, (x, new_c) = convlstm(inputs) # forward pass
        
        #print(output.shape)
        print('x.shape {}'.format(x.shape))
        print('clouds.shape {}'.format(clouds.shape))
        loss = loss_fn(x, clouds)
        loss.backward() # bakward step 
        optimizer.step() # optimize step
        
        # print statistics
        running_loss += loss.item()
        #if i % 2000 == 1999:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss))
        #running_loss = 0.0
        score.append(running_loss)
    
    # Understanding plotting. 
    #plt.scatter(epoch, running_loss)
    #plt.show()
print('Finished Training')

In [ ]:
nbr_epochs = 10

In [ ]:
plt.scatter(np.arange(nbr_epochs), score)
plt.show()

In [14]:
def Xavier_init_weights(m):
    """ glorot initialization """
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)
        
def LeCunn_init_weight(m):
    stdv = 1. / math.sqrt(m.weight.size(1))
    m.weight.data.uniform_(-stdv, stdv)
    if m.bias is not None:
        m.bias.data.uniform_(-stdv, stdv)

In [7]:
class ConvLSTMCell(nn.Module):
    """ Inspired by https://github.com/automan000/Convolution_LSTM_PyTorch. """
    def __init__(self, input_channels, hidden_channels, kernel_size, GPU = False):
        super(ConvLSTMCell, self).__init__()

        #assert hidden_channels % 2 == 0 # figure out what this does

        self.input_channels = input_channels
        self.hidden_channels = hidden_channels
        self.kernel_size = kernel_size
        self.num_features = 4

        self.padding = int((kernel_size - 1) / 2)

        self.Wxi = nn.Conv2d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
        self.Whi = nn.Conv2d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)
        self.Wxf = nn.Conv2d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
        self.Whf = nn.Conv2d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)
        self.Wxc = nn.Conv2d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
        self.Whc = nn.Conv2d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)
        self.Wxo = nn.Conv2d(self.input_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=True)
        self.Who = nn.Conv2d(self.hidden_channels, self.hidden_channels, self.kernel_size, 1, self.padding, bias=False)
        # cell 
        self.Wci = None # input to sigmoid
        self.Wcf = None # 
        self.Wco = None

    def forward(self, x, h, c):
        ci = torch.sigmoid(self.Wxi(x) + self.Whi(h) + c * self.Wci)
        cf = torch.sigmoid(self.Wxf(x) + self.Whf(h) + c * self.Wcf)
        cc = cf * c + ci * torch.tanh(self.Wxc(x) + self.Whc(h))
        co = torch.sigmoid(self.Wxo(x) + self.Who(h) + cc * self.Wco)
        ch = co * torch.tanh(cc)
        return ch, cc

    def init_hidden(self, batch_size, hidden, shape):
        if self.Wci is None:
            self.Wci = Variable(torch.zeros(1, hidden, shape[0], shape[1])).cpu() #cuda() here
            self.Wcf = Variable(torch.zeros(1, hidden, shape[0], shape[1])).cpu()#.cuda()
            self.Wco = Variable(torch.zeros(1, hidden, shape[0], shape[1])).cpu()#.cuda()
        else:
            assert shape[0] == self.Wci.size()[2], 'Input Height Mismatched!'
            assert shape[1] == self.Wci.size()[3], 'Input Width Mismatched!'
        return (Variable(torch.zeros(batch_size, hidden, shape[0], shape[1])).cpu(), #cuda(),
                Variable(torch.zeros(batch_size, hidden, shape[0], shape[1])).cpu() )#cuda())


class ConvLSTM(nn.Module):
    # input_channels corresponds to the first input feature map
    # hidden state is a list of succeeding lstm layers.
    def __init__(self, input_channels, hidden_channels, kernel_size, step=1, effective_step=[1]):
        super(ConvLSTM, self).__init__()
        self.input_channels = [input_channels] + hidden_channels
        self.hidden_channels = hidden_channels
        self.kernel_size = kernel_size
        self.num_layers = len(hidden_channels)
        self.step = step
        self.effective_step = effective_step
        self._all_layers = []
        for i in range(self.num_layers):
            name = 'cell{}'.format(i)
            cell = ConvLSTMCell(self.input_channels[i], self.hidden_channels[i], self.kernel_size)
            setattr(self, name, cell)
            self._all_layers.append(cell)

    def forward(self, inputs):
        internal_state = []
        outputs = []
        for step in range(self.step):
            x = inputs
            for i in range(self.num_layers):
                # all cells are initialized in the first step
                name = 'cell{}'.format(i)
                
                if step == 0:
                    bsize, _, height, width = x.size()
                    (h, c) = getattr(self, name).init_hidden(batch_size=bsize, hidden=self.hidden_channels[i],
                                                             shape=(height, width))
                    internal_state.append((h, c))

                # do forward
                (h, c) = internal_state[i]
                x, new_c = getattr(self, name)(x, h, c)
                internal_state[i] = (x, new_c)
            # only record effective steps
            if step in self.effective_step:
                outputs.append(x)

        return outputs, (x, new_c)

In [14]:
train, test = create_ML_repo()

# SET UP MODEL

In [ ]:
sns.heatmap(diff[0][0][:][:])